In [ ]:
# 1️⃣ Verificar GPU disponible
import torch
print(f"PyTorch versión: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memoria GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU no detectada. Activa GPU en: Runtime → Change runtime type")

In [ ]:
# 2️⃣ Instalar Ultralytics (YOLOv8)
!pip install ultralytics -q
print("✅ Ultralytics instalado")

In [ ]:
# 3️⃣ Subir archivo ZIP del dataset
# OPCIÓN A: Subir desde tu PC
from google.colab import files
import os

print("📤 Sube el archivo EPP.v1i.yolov8.zip desde tu PC")
uploaded = files.upload()

# Verificar archivo subido
if 'EPP.v1i.yolov8.zip' in uploaded:
    print(f"✅ Archivo subido: {len(uploaded['EPP.v1i.yolov8.zip']) / 1e6:.2f} MB")
else:
    print("❌ No se encontró EPP.v1i.yolov8.zip")
    print("   Asegúrate de que el archivo se llame exactamente: EPP.v1i.yolov8.zip")

In [ ]:
# 4️⃣ Extraer dataset
import zipfile

print("📦 Extrayendo dataset...")
with zipfile.ZipFile('EPP.v1i.yolov8.zip', 'r') as zip_ref:
    zip_ref.extractall('EPP_dataset')

print("✅ Dataset extraído")

# Verificar estructura
!ls -la EPP_dataset/

In [ ]:
# 5️⃣ Verificar data.yaml
!cat EPP_dataset/data.yaml

In [ ]:
# 6️⃣ Corregir rutas en data.yaml (si es necesario)
import yaml

# Leer data.yaml
with open('EPP_dataset/data.yaml', 'r') as f:
    data = yaml.safe_load(f)

# Corregir rutas relativas
data['train'] = 'train/images'
data['val'] = 'train/images'  # Usar train como validación si no hay carpeta valid
data['test'] = 'train/images'

# Guardar
with open('EPP_dataset/data.yaml', 'w') as f:
    yaml.dump(data, f)

print("✅ data.yaml actualizado")
print(f"\nClases detectables ({data['nc']}):")
for i, name in enumerate(data['names']):
    print(f"  {i}: {name}")

In [ ]:
# 7️⃣ Entrenar el modelo
from ultralytics import YOLO

print("🚀 Iniciando entrenamiento...\n")

# Cargar modelo base
model = YOLO('yolov8n.pt')

# Entrenar
results = model.train(
    data='EPP_dataset/data.yaml',
    epochs=50,              # 50 épocas (ajusta según necesites)
    imgsz=640,              # Tamaño de imagen
    batch=16,               # Batch size
    device=0,               # GPU
    project='runs/ppe',     # Carpeta de resultados
    name='train',           # Nombre del experimento
    patience=10,            # Early stopping
    save=True,              # Guardar modelo
    plots=True,             # Generar gráficas
    verbose=True
)

print("\n✅ Entrenamiento completado")

In [ ]:
# 8️⃣ Verificar resultados
!ls -lh runs/ppe/train/weights/

print("\n📊 Modelos generados:")
print("  - best.pt: Mejor modelo (menor loss)")
print("  - last.pt: Último checkpoint")

In [ ]:
# 9️⃣ Ver métricas de entrenamiento
from IPython.display import Image, display

print("📈 Curvas de entrenamiento:")
display(Image('runs/ppe/train/results.png'))

print("\n🔍 Matriz de confusión:")
display(Image('runs/ppe/train/confusion_matrix.png'))

In [ ]:
# 🔟 Probar el modelo entrenado
from PIL import Image
import matplotlib.pyplot as plt

# Cargar mejor modelo
trained_model = YOLO('runs/ppe/train/weights/best.pt')

# Buscar una imagen de prueba
import glob
test_images = glob.glob('EPP_dataset/train/images/*.jpg')[:3]

print(f"🧪 Probando con {len(test_images)} imágenes...\n")

for img_path in test_images:
    # Predecir
    results = trained_model(img_path)
    
    # Mostrar
    plt.figure(figsize=(10, 10))
    plt.imshow(results[0].plot())
    plt.axis('off')
    plt.title(f"Detecciones en {img_path.split('/')[-1]}")
    plt.show()
    
    # Mostrar detecciones
    if len(results[0].boxes) > 0:
        print(f"  ✅ {len(results[0].boxes)} objeto(s) detectado(s)")
        for box in results[0].boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            name = trained_model.names[cls]
            print(f"     - {name}: {conf:.2%}")
    else:
        print("  ℹ️  No se detectaron objetos")
    print()

In [ ]:
# 1️⃣1️⃣ Descargar modelo entrenado
from google.colab import files
import shutil

# Copiar y renombrar
shutil.copy('runs/ppe/train/weights/best.pt', 'ppe_best.pt')

print("📥 Descargando modelo entrenado...")
files.download('ppe_best.pt')

print("\n✅ Modelo descargado: ppe_best.pt")
print("\n📝 Siguiente paso:")
print("  1. Copia ppe_best.pt a: API/models/")
print("  2. Configura en .env:")
print("     MODEL_PATH=models/ppe_best.pt")
print("     USE_ROBOFLOW=False")
print("  3. Ejecuta: python main.py")

## 🎉 ¡Entrenamiento Completado!

### 📊 Resultados
- Modelo entrenado: `ppe_best.pt`
- Gráficas en: `runs/ppe/train/`

### 🔄 Próximos Pasos
1. Descarga `ppe_best.pt`
2. Colócalo en `API/models/`
3. Actualiza `.env`
4. Inicia la API: `python main.py`

### 🔧 Si quieres mejorar el modelo
- Aumenta `epochs` (ej: 100)
- Usa modelo más grande: `yolov8s.pt` o `yolov8m.pt`
- Ajusta `batch` según memoria GPU

### 📝 Clases detectadas
1. Botas
2. Casco
3. Chaleco
4. Gafas
5. Guantes
6. Protector
7. Tapabocas